In [1]:
!pip install psycopg2

  Using cached psycopg2-2.7.3.1-cp36-cp36m-manylinux1_x86_64.whl


In [2]:
import psycopg2
from psycopg2.extras import RealDictCursor

In [35]:
def execute_query(query, update = False):
    HOST = 'postgres'
    DBNAME = 'postgres'
    USER = 'postgres'
    
    con = psycopg2.connect(host=HOST, dbname = DBNAME, user = USER)
    curs = con.cursor(cursor_factory=RealDictCursor)
    curs.execute(query)
    if not update:
        results = curs.fetchall()
        con.close()
        return results
    con.close()
    return None

In [36]:
query = 'SELECT * FROM pages'

In [37]:
execute_query(query)

[]

In [38]:
fake_page = (1, 'machine learning', 'this is a fake page about machine learning')

In [39]:
', '.join([str(i) for i in fake_page])

'1, machine learning, this is a fake page about machine learning'

In [40]:
def format_data_for_query(data_list):
    new_data_list = []
    for i in data_list:
        if type(i) == str:
            new_data_list.append("'{}'".format(i))
        else:
            new_data_list.append(str(i))
    return ', '.join(new_data_list)

In [41]:
format_data_for_query(fake_page)

"1, 'machine learning', 'this is a fake page about machine learning'"

In [42]:
def update_table(table_name, col_values):
    query = '''
            BEGIN;
            INSERT INTO {} VALUES ({});
            COMMIT;
            '''.format(table_name, format_data_for_query(col_values))
    execute_query(query, update=True)

In [43]:
def insert_1(page_id, title, text, category):
    update_table('pages', [page_id, title])
    update_table('page_content', [page_id, text])
    update_table('page_categories', [page_id, category])

In [52]:
insert_1(3, 'machine learning','this is a fake page about machine learning', 'macine learning')

In [45]:
execute_query('SELECT * FROM page_categories')

[{'category': 'macine learning', 'page_id': 3}]

In [46]:
def purge_table(table_name):
    query = '''
            BEGIN;
            DELETE FROM {};
            COMMIT;
            '''.format(table_name)
    execute_query(query, update=True)

In [48]:
purge_table('pages')
purge_table('page_content')
purge_table('page_categories')

In [53]:
execute_query('SELECT * FROM pages')

[{'page_id': 3, 'title': 'machine learning'}]

In [54]:
execute_query('SELECT * FROM page_content')

[{'content': 'this is a fake page about machine learning', 'page_id': 3}]

In [55]:
execute_query('SELECT * FROM page_categories')

[{'category': 'macine learning', 'page_id': 3}]

In [66]:
query = '''
          SELECT * FROM pages p
          JOIN page_categories c ON 
          p.page_id = c.page_id
          JOIN page_content pc ON
          p.page_id = pc.page_id
          '''

In [67]:
execute_query(query)

[{'category': 'macine learning',
  'content': 'this is a fake page about machine learning',
  'page_id': 3,
  'title': 'machine learning'}]